In [12]:
import numpy as np
from scipy.stats import chi2, chi2_contingency

In [ ]:
def estimate_test_size(m=95, n=100, pi=0.5, alpha=0.1, num_simulations=10000):
    """
    Estimate the actual size of the chi-squared test for H0: p = q.
    
    Parameters:
    - m, n: Sample sizes
    - alpha: Nominal significance level
    - pi: True probability under H0 (p = q = pi)
    - num_simulations: Number of simulated datasets
    
    Returns:
    - empirical_size: Proportion of times H0 is rejected
    """
    rejections = 0 

    for _ in range(num_simulations):
        # Simulate data
        S_x = np.random.binomial(m, pi)
        S_y = np.random.binomial(n, pi)
        S = S_x + S_y
        F_x = m - S_x
        F_y = n - S_y
        F = F_x + F_y
        E_Sx = m * S / (m + n)
        E_Sy = n * S / (m + n)
        E_Fx = m - E_Sx
        E_Fy = n - E_Sy

        # Compute test statistic
        # test_statistic = (m + n)*((S_x * F_y) - (S_y * F_x))**2 / (m * n * S * F)
        test_statistic = (S_x - E_Sx)**2 / E_Sx + (S_y - E_Sy)**2 / E_Sy + (F_x - E_Fx)**2 / E_Fx + (F_y - E_Fy)**2 / E_Fy

        # Compute critical value for chi-squared distribution
        critical_value = np.percentile(np.random.chisquare(1, num_simulations), 100 * (1 - alpha))
        # critical_value = chi2.ppf(1 - alpha, 1)  # Uncomment if scipy is available

        # Check if we reject H0
        if test_statistic > critical_value:
            rejections += 1

    # Calculate empirical size
    empirical_size = rejections / num_simulations
    return empirical_size


This function above is doing to many approximations with all the calculations. I will use Scipy to do the calculations.

In [17]:
def estimate_test_size1(m=95, n=100, pi=0.5, alpha=0.1, num_simulations=10000):
    rejections = 0 

    for _ in range(num_simulations):
        # Simulate data
        S_x = np.random.binomial(m, pi)
        S_y = np.random.binomial(n, pi)
        F_x = m - S_x
        F_y = n - S_y

        # Build the contingency table
        table = np.array([[S_x, F_x],
                        [S_y, F_y]])

        # Get the test statistic from scipy
        chi2_stat, p_value, dof, expected = chi2_contingency(table, correction=False)

        # Check if we reject H0
        if p_value < alpha:
            rejections += 1
    
    # return empirical size
    return rejections / num_simulations
    

In [18]:
m = 95
n = 100
pi = np.linspace(0.1, 0.9, 9)
alpha = 0.1
num_simulations = 10000

empirical_sizes = np.zeros(len(pi))
for i in range(len(pi)):
    empirical_sizes[i] = estimate_test_size1(m, n, pi[i], alpha, num_simulations)
    print(f"True probability (pi): {pi[i]:.2f}, Empirical size: {empirical_sizes[i]:.4f}")


True probability (pi): 0.10, Empirical size: 0.0993
True probability (pi): 0.20, Empirical size: 0.1034
True probability (pi): 0.30, Empirical size: 0.0970
True probability (pi): 0.40, Empirical size: 0.0915
True probability (pi): 0.50, Empirical size: 0.0970
True probability (pi): 0.60, Empirical size: 0.0909
True probability (pi): 0.70, Empirical size: 0.1002
True probability (pi): 0.80, Empirical size: 0.1003
True probability (pi): 0.90, Empirical size: 0.1054
